# Regression on Yb based OLED dataset by LightAutoML

Designed by Koshelev Daniil

https://github.com/Lamblador/Yb_OLED_Dataset

This noteboot show how to read and use YbOLED dataset with LightAutoML package.

# Import and installation of modules

In [1]:
!pip install lightautoml
from IPython.display import clear_output
clear_output()

In [2]:
import pandas as pd
import numpy as np
import torch
import sklearn

In [226]:
!git clone https://github.com/Lamblador/Yb_OLED_Dataset/

Cloning into 'Yb_OLED_Dataset'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 16 (delta 3), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (16/16), 49.00 KiB | 1.09 MiB/s, done.
Resolving deltas: 100% (3/3), done.


# Initializing of the LightAutoML

In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Standard python libraries
import os
import requests

# Essential DS libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import torch

# LightAutoML presets, task and report generation
from lightautoml.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML
from lightautoml.tasks import Task
from lightautoml.report.report_deco import ReportDeco, ReportDecoUtilized
from lightautoml.addons.tabular_interpretation import SSWARM

Standart parameters

In [228]:
N_THREADS = 4 # number of the CPU to use
N_FOLDS = 5 # number folds in cross-validation
RANDOM_STATE = 42 # random state
TEST_SIZE = 0.2 # test dataset size
TIMEOUT = 300 # time for model to train
TARGET_NAME = 'ECE  uW/W' #name of the target parameter
# you can chose 'ECE  uW/W', 'Max. irradiance  uW/cm2', 'Uon ' or 'EQE  %'

In [229]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

In [230]:
task = Task('reg') # tast regression

### Read dataset

In [231]:
data = pd.read_csv('/content/Yb_OLED_Dataset/yb_oled_data_short2.csv', delimiter=';') # read file
data.drop('year', axis=1, inplace=True) #drop year data - no information for ML
data.dropna(subset = [TARGET_NAME], inplace=True) # drop all rows without target data

data.head()

,HIL tikness nm,HIL HOMO,HIL LUMO,HTL tikness nm,HTL HOMO,HTL LUMO,HTL HOMO-EML HOMO,HTL LUMO-EML LUMO,EML tikness nm,EML HOMO,...,Max. irradiance uW/cm2,EQE %,ECE uW/W,Uon,t us,QY %,hole mobilty cm2/Vs,electron mobility cm2/Vs,total cm2/Vs,Pixel size mm2
5,30,-5.5,-2.4,0,NaN,NaN,NaN,NaN,50,NaN,...,0.80,NaN,0.1,10.0,24.0,1.20,NaN,NaN,NaN,NaN
8,25,-5.2,-2.4,0,NaN,NaN,0.72,0.26,40,-5.92,...,22.48,NaN,51.0,7.6,NaN,2.40,6.240000e-08,2.690000e-12,3.210000e-06,NaN
9,25,-5.2,-2.4,0,NaN,NaN,0.72,0.26,40,-5.92,...,12.13,NaN,44.0,7.7,NaN,1.41,NaN,NaN,2.690000e-05,NaN
10,25,-5.2,-2.4,0,NaN,NaN,0.72,0.26,40,-5.92,...,9.60,NaN,42.0,7.9,NaN,1.33,NaN,NaN,1.300000e-07,NaN
11,25,-5.5,-2.4,0,NaN,NaN,0.42,0.26,40,-5.92,...,19.29,NaN,47.0,7.7,NaN,1.92,NaN,NaN,4.500000e-08,NaN


Train/test split

In [232]:
train_data, test_data = train_test_split(
    data,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE
)

print(f'Data is splitted. Parts sizes: train_data = {train_data.shape}, test_data = {test_data.shape}')

train_data.head()

Data is splitted. Parts sizes: train_data = (21, 38), test_data = (6, 38)


,HIL tikness nm,HIL HOMO,HIL LUMO,HTL tikness nm,HTL HOMO,HTL LUMO,HTL HOMO-EML HOMO,HTL LUMO-EML LUMO,EML tikness nm,EML HOMO,...,Max. irradiance uW/cm2,EQE %,ECE uW/W,Uon,t us,QY %,hole mobilty cm2/Vs,electron mobility cm2/Vs,total cm2/Vs,Pixel size mm2
36,50,-5.2,-2.3,20,-5.2,-2.3,0.627,0.895,30,-5.827,...,17.00,0.06000,188.0,4.5,14.2,0.91,6.470000e-05,6.080000e-05,NaN,12.0
37,50,-5.2,-2.3,20,-5.2,-2.3,0.691,0.947,30,-5.891,...,7.00,0.12000,429.0,4.5,14.4,0.95,1.550000e-04,1.010000e-04,NaN,12.0
32,50,-5.2,-2.3,15,-5.8,-2.2,-0.662,0.450,23,-5.138,...,4.00,0.00030,93.0,3.9,10.0,0.50,NaN,NaN,NaN,12.0
46,50,-5.2,-2.3,20,-5.2,-2.3,NaN,NaN,23,NaN,...,16.00,0.00045,140.0,4.0,NaN,0.80,NaN,NaN,NaN,12.0
8,25,-5.2,-2.4,0,NaN,NaN,0.720,0.260,40,-5.920,...,22.48,NaN,51.0,7.6,NaN,2.40,6.240000e-08,2.690000e-12,0.000003,NaN


Roles of the columns. **target** - target value to predict, **drop** - ignoragle value.

In [233]:
#TARGET_NAME = 'Max. irradiance  uW/cm2' #Max. irradiance  uW/cm2 'ECE  uW/W', 'EQE  %', 'Uon'
roles = {
    'target': TARGET_NAME,
    'drop': ['Max. irradiance  uW/cm2', 'EQE  %', 'Uon'] # HAVE TO CHANGE list if you change TARGET_NAME
}

In [234]:
automl = TabularAutoML( #tabular auto ml pipeline class
    task = task,
    timeout = TIMEOUT,
    cpu_limit = N_THREADS,
    reader_params = {'n_jobs': N_THREADS, 'cv': N_FOLDS, 'random_state': RANDOM_STATE},
)

## Model training

In [235]:
%%time
out_of_fold_predictions = automl.fit_predict(train_data, roles = roles, verbose = 1)

[15:05:25] Stdout logging level is INFO.


INFO:lightautoml.automl.presets.base:Stdout logging level is INFO.


[15:05:25] Task: reg



INFO:lightautoml.automl.presets.base:Task: reg



[15:05:25] Start automl preset with listed constraints:


...

INFO:lightautoml.ml_algo.base:Fitting Lvl_0_Pipe_1_Mod_2_CatBoost finished. score = -13496.901950175428


[15:05:49] Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed


INFO:lightautoml.ml_algo.base:Lvl_0_Pipe_1_Mod_2_CatBoost fitting and predicting completed


[15:05:49] Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 208.00 secs


INFO:lightautoml.ml_algo.tuning.optuna:Start hyperparameters optimization for Lvl_0_Pipe_1_Mod_3_Tuned_CatBoost ... Time budget is 208.00 secs
INFO:optuna.storages._in_memory:A new study created in memory with name: no-name-b8ce2ba2-101c-4e85-ab67-5a76a1c4f483
INFO3:lightautoml.ml_algo.boost_cb:0:	learn: 97.2782733	test: 162.2731881	best: 162.2731881 (0)	total: 128us	remaining: 258ms
DEBUG:lightautoml.ml_algo.boost_cb:100:	learn: 94.9712357	test: 156.2761517	best: 156.2761517 (100)	total: 7.71ms	remaining: 145ms
DEBUG:lightautoml.ml_algo.boost_cb:200:	learn: 94.9625076	test: 156.1641004	best: 156.1641004 (200)	total: 13.7ms	remaining: 123ms
DEBUG:lightautoml.ml_algo.boost_cb:300:	learn: 94.9624664	test: 156.1618839	best: 156.1618839 (300)	total: 21.2ms	remaining: 119ms
DEBUG:lightautoml.ml_algo.boost_cb:400:	learn: 94.9624662	test: 156.1618400	best: 156.1618400 (400)	total: 28.8ms	remaining: 115ms
DEBUG:lightautoml.ml_algo.boost_cb:500:	learn: 94.9624662	test: 156.1618391	best: 156.161

## Model prediction

In [236]:
%%time

test_predictions = automl.predict(test_data)
print(f'Prediction for test_data:\n{test_predictions}\nShape = {test_predictions.shape}')

In [237]:
y_true = test_data[TARGET_NAME]
y_pred = test_predictions.data
y_true

In [238]:
print(automl.create_model_str_desc())

## Model prediction accuracy by MAPE

In [239]:
from sklearn.metrics import mean_absolute_percentage_error
scorer = mean_absolute_percentage_error(y_true, y_pred)
print(scorer)

## Feature analysis

In [ ]:
%%time

# Accurate feature importances calculation with detailed info (Permutation importances) -  can take long time to calculate
accurate_fi = automl.get_feature_scores('accurate', test_data, silent = True)

In [ ]:
accurate_fi.set_index('Feature')['Importance'].plot.bar(figsize = (30, 10), grid = True)

# Model saving

In [ ]:
import joblib
joblib.dump(automl, 'model.pkl') #model save
#automl=joblib.load(‘model.pkl’) #model load